In [277]:
import pandas as pd
import numpy as np
import datetime

In [377]:
budget_23 = pd.concat(
    pd.read_excel("TALOUSARVIO 23.xlsx", sheet_name=None, header=[ 10, 11]),
    ignore_index=True,
)
if len(budget_23.columns) != 31:
    raise Exception("Sheets don't follow the same pattern")
# Dropping columns we don't need
budget_23 = budget_23.drop(
    budget_23.columns[[3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]],
    axis=1,
)
# replace yes and no in the housing column
budget_23.iloc[:, 2] = budget_23.iloc[:, 2].map({"K": True, "E": False})


# CHANGE THIS HERE TO ONLY GET PROJECTS AND NOT MISS ANY PROJECT
budget_23 = budget_23[budget_23[budget_23.columns[15]].notna()]
budget_23 = budget_23.replace({np.nan: None, "?": None})
currYear = datetime.date.today().year
financeShift = list(budget_23.columns.get_level_values(0))[4] - currYear - 1
if financeShift != 0:
    finance_df = budget_23.iloc[:, 4:14]
    budget_23.iloc[:, 4:14] = finance_df.shift(financeShift, axis=1)
budget_23


Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                    Unnamed: 0_level_1 Unnamed: 1_level_1   
16       Siltasaarenkatu (Toinen linja - Kolmas linja)                 K1   
19   Hakaniementori + (Länsipää Miina Sillanpään katu)               K5.1   
20             Siltasaarenkatu (Hakaniemenr - Hämeent)               K5.1   
21          Signe Branderin terassi (itä+länsi) + kuja               K5.1   
22                                  Näkinkulku (pinta)               K5.1   
..                                                 ...                ...   
481                                    Vaskipellonkuja                 K4   
483                                       Vankkurikuja                 K4   
484                                           Iespolku                 K4   
487                                   Vaskipellonpolku                 K4   
489                                        Kesantokuja                 K4   

    asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028   2029  \
       Kyllä / Ei  1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 €   
16           True  1000.0   None   None   None   None   None   None   None   
19          False  3000.0   None   None   None   None   None   None   None   
20           True   500.0   None   None   None   None   None   None   None   
21           True  3500.0   None   None   None   None   None   None   None   
22           True   300.0   None   None   None   None   None   None   None   
..            ...     ...    ...    ...    ...    ...    ...    ...    ...   
481          None    None   None   None   None   None   None   None   None   
483          None    None   None   None   None   None   None   None   None   
484          None    None   None   None   None   None   None   None   None   
487          None    None   None   None   None   None   None   None   None   
489          None    None   None   None   None   None   None   None   None   

      2030   2031   2032                                                     \
    1000 € 1000 € 1000 €                                           1000 €.1   
16    None   None   None  . 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA...   
19    None   None   None    Suunnittelun aloitus vasta 25 ja toteutus 26+27   
20    None   None   None                                               None   
21    None   None   None                                               None   
22    None   None   None                                               None   
..     ...    ...    ...                                                ...   
481   None   None   None                                               None   
483   None   None   None                                               None   
484   None   None   None                                               None   
487   None   None   None                                               None   
489   None   None   None                                               None   

       HKR / PW  
    HANKENUMERO  
16         2637  
19          191  
20          839  
21         3174  
22         3132  
..          ...  
481        1507  
483        1509  
484        1510  
487        1503  
489        1515  

[225 rows x 16 columns]

In [392]:
plan_23 = pd.concat(
    pd.read_excel("TOIMINTASUUNNITELMA 23.xlsx", sheet_name=None, header=[0, 1]),
    ignore_index=True,
)
if len(plan_23.columns) != 28:
    raise Exception("Sheets don't follow the same pattern")

plan_23 = plan_23.drop(
    plan_23.columns[
        [5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
    ],
    axis=1,
)

# Replace all " with the sap numbers before it
plan_23.iloc[:, 1] = plan_23.iloc[:, 1].replace(["\""], method="ffill")
plan_23.iloc[:, 2] = plan_23.iloc[:, 2].replace(["\""], method="ffill")

# Dropping all non project rows
plan_23 = plan_23[plan_23[plan_23.columns[1]].notna() | plan_23[plan_23.columns[2]].notna() | plan_23[plan_23.columns[3]].notna()| plan_23[plan_23.columns[4]].notna() | plan_23[plan_23.columns[7]].notna()]
plan_23 = plan_23.replace({np.nan: None, "?": None})

# plan_23.groupby(plan_23.columns[7]).aggregate({plan_23.columns[1]: lambda x: list(x)})
merged_Frame = pd.merge(budget_23, plan_23, left_on=[budget_23.columns[15]], right_on=[plan_23.columns[7]], how='inner')

merged_Frame

Unnamed: 0_level_0 Unnamed: 1_level_0  \
                                  Unnamed: 0_level_1 Unnamed: 1_level_1   
0      Siltasaarenkatu (Toinen linja - Kolmas linja)                 K1   
1                     Merihaan täydennysrakentaminen               K5.1   
2                     Merihaan täydennysrakentaminen               K5.1   
3                     Merihaan täydennysrakentaminen               K5.1   
4                     Merihaan täydennysrakentaminen               K5.1   
...                                              ...                ...   
17868                                   Smoltinkaari               K5.1   
17869                                    Smoltinkuja               K5.1   
17870                                   Smoltinkulku               K5.1   
17871      Siltalanpuiston kaava-alue, ak 12243 (AM)               K5.1   
17872                               Kiertotähdenkuja               K5.1   

      asuntotuotan. ennuste   2023   2024   2025   2026   2027   2028  ...  \
         Kyllä / Ei  1000 € 1000 € 1000 € 1000 € 1000 € 1000 € 1000 €  ...   
0              True  1000.0   None   None   None   None   None   None  ...   
1              None  1000.0   None   None   None   None   None   None  ...   
2              None  1000.0   None   None   None   None   None   None  ...   
3              None  1000.0   None   None   None   None   None   None  ...   
4              None  1000.0   None   None   None   None   None   None  ...   
...             ...     ...    ...    ...    ...    ...    ...    ...  ...   
17868          True    None   None   None   None   None   None   None  ...   
17869          True    None   None   None   None   None   None   None  ...   
17870          True    None   None   None   None   None   None   None  ...   
17871          None    None  100.0  100.0  100.0  100.0   None   None  ...   
17872          True   400.0   None   None   None   None   None   None  ...   

        2032                                                       HKR / PW  \
      1000 €                                           1000 €.1 HANKENUMERO   
0       None  . 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA...        2637   
1       None                                               None        None   
2       None                                               None        None   
3       None                                               None        None   
4       None                                               None        None   
...      ...                                                ...         ...   
17868   None                                               None        2032   
17869   None                                               None        2033   
17870   None                                               None        2034   
17871   None                                               None          10   
17872   None                                               None        1015   

                                  Unnamed: 0_level_0  LASKE-SAP-  Verkko-  \
                                               HANKE    projekti   numero   
0      Siltasaarenkatu 2-3 linja (Siltasaarenportti)  2814I00709  6703970   
1                                       Puutarhakuja  2814I00019  6705806   
2                                Mannerheimintie 3-5        None     None   
3                    Kaartinkortteli, Fabianinkatu 2        None     None   
4                                      Keskustan THK        None     None   
...                                              ...         ...      ...   
17868                                   Smoltinkaari  2814I01528  6701967   
17869                                    Smoltinkuja  2814I01528  6701967   
17870                                   Smoltinkulku  2814I01528  6701967   
17871                Siltalanpuiston kaava-alue (AM)  2814I01520  6701064   
17872                               Kiertotähdenkuja  2814I01535  6705904   

            A

In [384]:
plan_23

,Unnamed: 0_level_0,LASKE-SAP-,Verkko-,Alue-,Kohteen,Hankkeen,Huom.,HKR / PW
,HANKE,projekti,numero,vastaava,projektipäällikkö,TS2023,Unnamed: 26_level_1,HANKENUMERO
11,Puutarhakuja,2814I00019,6705806,None,Kimari,140,suunnitelmat 3/23,None
12,Mannerheimintie 3-5,None,None,None,Kimari,30,pienet järjestelyt hotellin edustalla,None
14,"Kaartinkortteli, Fabianinkatu 2",None,None,None,NN,270,None,None
16,Keskustan THK,None,None,Kaalikoski,None,50,kaavasta valitettu,None
17,Keskustan THK liittyvät kadut,None,None,None,Kaalikoski,50,tarvitaan maisemaark. näkemystä mukaan aukion ...,None
...,...,...,...,...,...,...,...,...
724,Vanhankaupunginkoski ja Kuninkaansaari,2814I03214,6704221,None,Luomanen,None,None,None
726,Pyörökivenpuiston tulvasuojelu,2814I03210,6703192,None,NN,50,None,None
727,"Idän rantareitti, perusparannus v. Reginankuja...",2814I03216,6705620,None,Knuuti,540,None,None


In [259]:
budget_23_sp3_data = [
    {
        "name": name,
        "category": category,
        "effectHousing": effectHousing,
        "costForecast": costForecast,
        "budgetProposalCurrentYearPlus1": bpCPlus1,
        "budgetProposalCurrentYearPlus2": bpCPlus2,
        "preliminaryCurrentYearPlus3": pcyPlus3,
        "preliminaryCurrentYearPlus4": pcyPlus4,
        "preliminaryCurrentYearPlus5": pcyPlus5,
        "preliminaryCurrentYearPlus6": pcyPlus6,
        "preliminaryCurrentYearPlus7": pcyPlus7,
        "preliminaryCurrentYearPlus8": pcyPlus8,
        "preliminaryCurrentYearPlus9": pcyPlus9,
        "preliminaryCurrentYearPlus10": pcyPlus10,
        "note_content": note_content,
        "hkrId": hkrId,
    }
    for name, category, effectHousing, costForecast, bpCPlus1, bpCPlus2, pcyPlus3, pcyPlus4, pcyPlus5, pcyPlus6, pcyPlus7, pcyPlus8, pcyPlus9, pcyPlus10, note_content, hkrId in zip(
        budget_23.iloc[:, 0],
        budget_23.iloc[:, 1],
        budget_23.iloc[:, 2],
        budget_23.iloc[:, 3],
        budget_23.iloc[:, 4],
        budget_23.iloc[:, 5],
        budget_23.iloc[:, 6],
        budget_23.iloc[:, 7],
        budget_23.iloc[:, 8],
        budget_23.iloc[:, 9],
        budget_23.iloc[:, 10],
        budget_23.iloc[:, 11],
        budget_23.iloc[:, 12],
        budget_23.iloc[:, 13],
        budget_23.iloc[:, 14],
        budget_23.iloc[:, 15],
    )
]

In [260]:
budget_23_sp3_data

[{'name': 'Siltasaarenkatu (Toinen linja - Kolmas linja)',
  'category': 'K1',
  'effectHousing': 'K',
  'costForecast': 1000.0,
  'budgetProposalCurrentYearPlus1': None,
  'budgetProposalCurrentYearPlus2': None,
  'preliminaryCurrentYearPlus3': None,
  'preliminaryCurrentYearPlus4': None,
  'preliminaryCurrentYearPlus5': None,
  'preliminaryCurrentYearPlus6': None,
  'preliminaryCurrentYearPlus7': None,
  'preliminaryCurrentYearPlus8': None,
  'preliminaryCurrentYearPlus9': None,
  'preliminaryCurrentYearPlus10': None,
  'note_content': '. 23 0,3 milj. Kivityös 24 0,5 milj.  TARKISTA VARAUS 24?',
  'hkrId': 2637},
 {'name': 'Hakaniementori + (Länsipää Miina Sillanpään katu)',
  'category': 'K5.1',
  'effectHousing': 'E',
  'costForecast': 3000.0,
  'budgetProposalCurrentYearPlus1': None,
  'budgetProposalCurrentYearPlus2': None,
  'preliminaryCurrentYearPlus3': None,
  'preliminaryCurrentYearPlus4': None,
  'preliminaryCurrentYearPlus5': None,
  'preliminaryCurrentYearPlus6': None,
  